In [2]:
import os
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from user_agent import generate_user_agent

In [3]:
# 對網頁做解析的程式方法
def crawler(url):
    time.sleep(5)
    # 製作對網頁請求的假資料，如：假裝是IOS手機、Android手機、mac瀏覽器、desktop
    user_agent = generate_user_agent(device_type='desktop')
    # 代理 ip
    # proxies = {
    #   'http': '123.205.24.240:80',
    # }
    # headers
    headers = {
    #User-Agent的值表示這個請求是由Chrome 51瀏覽器（User-Agent字符串中的信息）發送，而且是在一個Chrome OS x86_64的環境下
    'User-Agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
    #表示使用者首選的語言是美式英文（en-US），優先權為0.5。
    #若伺服器能夠提供美式英文內容，就會優先返回該內容，如果不行，則默認使用一般的英文內容（en）
    'Accept-Language': 'en-US, en;q=0.5'}

    '''headers={
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,',
        'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': "macOS",
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': user_agent,
    }'''

    #對指定的url連結做請求，並給予假資料
    r = requests.get(url, headers=headers)
    r.encoding='utf-8'
    #將網頁作解析，以利找到所需資料
    soup = BeautifulSoup(r.text, "html.parser")
    #回傳整個網頁的解析
    return soup

In [ ]:
browse_url = 'https://www.barnesandnoble.com/h/books/browse'
browse_soup = crawler(browse_url)

In [ ]:
First_Subject, First_Subject_url = [],[]
containers = browse_soup.find_all('section', class_ = 'links-container browse clearer')
def find_first_subject(container_num):
    for li in containers[container_num].find_all('li'):
        if li.find('a').get('href').split('/')[1] == 'b':
            if li.find('a').get('href').split('/')[4] == '_':
                First_Subject.append(li.find('a').get('href').split('/')[3])
                First_Subject_url.append(f'https://www.barnesandnoble.com{li.find('a').get('href')}')

want_container = [0,1,2,3,4,6]
for container_num in want_container:
    find_first_subject(container_num)

browse_df = pd.DataFrame({'First_Subject':First_Subject,'url':First_Subject_url})
browse_df = browse_df.drop_duplicates(ignore_index = True)

In [ ]:
First_Subject_2,Second_Subject,Second_Subject_url=[],[],[]

In [ ]:
for s_1_url in browse_df['url']:
    first_subject_soup = crawler(s_1_url)
    sidebar_sections = first_subject_soup.find_all('div',class_='sidebar__section refinements')
    
    if sidebar_sections == []:
        continue
    
    if sidebar_sections[0].find_all('li',class_='bg-gray-02 pt-xs pb-xs')!=[]:
        continue
    
    for sidebar_section in sidebar_sections:
        if sidebar_section.find('span', class_='augHeader').text == "\n Subjects\n\n":
            for li in sidebar_section.find_all('li'):
                    First_Subject_2.append(s_1_url.split('/')[5])
                    Second_Subject.append(li.text)
                    Second_Subject_url.append(f'https://www.barnesandnoble.com{li.find('a').get('href')}')
            print(s_1_url.split('/')[5])
        break
        

second_df = pd.DataFrame({'First_Subject_2':First_Subject_2,
                          'Second_Subject':Second_Subject,
                          'Second_Subject_url':Second_Subject_url})
second_df = second_df.drop_duplicates(ignore_index = True)

In [ ]:
First_Subject_3, Second_Subject_3, Third_Subject, url_3 = [],[],[],[]

In [ ]:
for s_2_url in second_df['Second_Subject_url']:
    
    if s_2_url.split('/')[5] == 'awards':
        continue

    second_subject_soup = crawler(s_2_url)
    sidebar_sections = second_subject_soup.find_all('div',class_='sidebar__section refinements')

    if sidebar_sections == []:
        First_Subject_3.append(s_2_url.split('/')[5])
        Second_Subject_3.append(s_2_url.split('/')[6])
        Third_Subject.append('_')
        url_3.append(s_2_url)
        print(f'{s_2_url.split('/')[5]}, {s_2_url.split('/')[6]}, _')
        continue
    
    if sidebar_sections[0].find_all('li',class_='bg-gray-02 pt-xs pb-xs')!=[]:
        continue
    
    if str(sidebar_sections).find('Subjects')==-1:
        First_Subject_3.append(s_2_url.split('/')[5])
        Second_Subject_3.append(s_2_url.split('/')[6])
        Third_Subject.append('_')
        url_3.append(s_2_url)
        print(f'{s_2_url.split('/')[5]}, {s_2_url.split('/')[6]}, _')
        continue

    for sidebar_section in sidebar_sections:
        if sidebar_section.find('span', class_='augHeader').text == "\n Subjects\n\n":
            for li in sidebar_section.find_all('li'):
                    First_Subject_3.append(s_2_url.split('/')[5])
                    Second_Subject_3.append(s_2_url.split('/')[6])
                    Third_Subject.append(li.find('a').get('href').split('/')[4])
                    url_3.append(f'https://www.barnesandnoble.com{li.find('a').get('href')}')
            print(f'{s_2_url.split('/')[5]}, {s_2_url.split('/')[6]}')
        
        break
        


third_df = pd.DataFrame({'First_Subject':First_Subject_3,
                          'Second_Subject':Second_Subject_3,
                          'Third_Subject':Third_Subject,
                          'url':url_3})
third_df = third_df.drop_duplicates(ignore_index = True)

In [ ]:
third_df.to_csv('third_df.csv', index = False)

In [ ]:
third_df = pd.read_csv('third_df.csv')
third_df

In [ ]:
first_subject, second_subject, third_subject, fourth_subject, url = [],[],[],[],[]
for row in third_df.index[4401:]:
  if third_df['Third_Subject'][row] == '_':
    first_subject.append(third_df['First_Subject'][row])
    second_subject.append(third_df['Second_Subject'][row])
    third_subject.append('_')
    fourth_subject.append('_')
    url.append(third_df['url'][row])
    print(f"{row}: {third_df['First_Subject'][row]}, {third_df['Second_Subject'][row]}, _")
    continue

  soup = crawler(third_df['url'][row])
  sidebar_sections = soup.find_all('div',class_='sidebar__section refinements')

  if sidebar_sections == []:
    first_subject.append(third_df['First_Subject'][row])
    second_subject.append(third_df['Second_Subject'][row])
    third_subject.append(third_df['Third_Subject'][row])
    fourth_subject.append('_')
    url.append(third_df['url'][row])
    print(f"{row}: {third_df['First_Subject'][row]}, {third_df['Second_Subject'][row]}, {third_df['Third_Subject'][row]}, _")
    continue

  if sidebar_sections[0].find_all('li',class_='bg-gray-02 pt-xs pb-xs')!=[]:
      continue

  if str(sidebar_sections).find('Subjects')==-1:
    first_subject.append(third_df['First_Subject'][row])
    second_subject.append(third_df['Second_Subject'][row])
    third_subject.append(third_df['Third_Subject'][row])
    fourth_subject.append('_')
    url.append(third_df['url'][row])
    print(f"{row}: {third_df['First_Subject'][row]}, {third_df['Second_Subject'][row]}, {third_df['Third_Subject'][row]}, _")
    continue

  for sidebar_section in sidebar_sections:
    if sidebar_section.find('span', class_='augHeader').text == "\n Subjects\n\n":
      for li in sidebar_section.find_all('li'):
        first_subject.append(third_df['First_Subject'][row])
        second_subject.append(third_df['Second_Subject'][row])
        third_subject.append(third_df['Third_Subject'][row])
        fourth_subject.append(li.find('a').get('href').split('/')[4])
        url.append(f"https://www.barnesandnoble.com{li.find('a').get('href')}")
      print(f"*{row}: {third_df['First_Subject'][row]}, {third_df['Second_Subject'][row]}, {third_df['Third_Subject'][row]}")

    break

'''fourth_df = pd.DataFrame({
    'First_Subject':first_subject,
    'Second_Subject':second_subject,
    'Third_Subject':third_subject,
    'Fourth_Subject':fourth_subject,
    'url':url
})

fourth_df = fourth_df.drop_duplicates(ignore_index = True)'''

In [ ]:
fourth_df = pd.DataFrame({
    'First_Subject':first_subject,
    'Second_Subject':second_subject,
    'Third_Subject':third_subject,
    'Fourth_Subject':fourth_subject,
    'url':url
})

fourth_df = fourth_df.drop_duplicates(ignore_index = True)

In [ ]:
fourth_df.to_csv('fourth_df_1.csv', index = False)

In [ ]:
fourth_df_1 = pd.read_csv('fourth_df_1.csv')
fourth_df_1

In [ ]:
first_subject, second_subject, third_subject, fourth_subject,fifth_subject, url =[], [],[],[],[],[]

In [ ]:

for row in fourth_df_1.index[13635:]:
  if fourth_df_1['Fourth_Subject'][row] == '_':
    first_subject.append(fourth_df_1['First_Subject'][row])
    second_subject.append(fourth_df_1['Second_Subject'][row])
    third_subject.append(fourth_df_1['Third_Subject'][row])
    fourth_subject.append(fourth_df_1['Fourth_Subject'][row])
    fifth_subject.append('_')
    url.append(fourth_df_1['url'][row])
    print(row)
    continue

  soup = crawler(fourth_df_1['url'][row])
  sidebar_sections = soup.find_all('div',class_='sidebar__section refinements')

  if sidebar_sections == []:
    first_subject.append(fourth_df_1['First_Subject'][row])
    second_subject.append(fourth_df_1['Second_Subject'][row])
    third_subject.append(fourth_df_1['Third_Subject'][row])
    fourth_subject.append(fourth_df_1['Fourth_Subject'][row])
    fifth_subject.append('_')
    url.append(fourth_df_1['url'][row])
    print(row)
    continue

  if sidebar_sections[0].find_all('li',class_='bg-gray-02 pt-xs pb-xs')!=[]:
      continue

  if str(sidebar_sections).find('Subjects')==-1:
    first_subject.append(fourth_df_1['First_Subject'][row])
    second_subject.append(fourth_df_1['Second_Subject'][row])
    third_subject.append(fourth_df_1['Third_Subject'][row])
    fourth_subject.append(fourth_df_1['Fourth_Subject'][row])
    fifth_subject.append('_')
    url.append(fourth_df_1['url'][row])
    print(row)

  for sidebar_section in sidebar_sections:
    if sidebar_section.find('span', class_='augHeader').text == "\n Subjects\n\n":
      for li in sidebar_section.find_all('li'):
        first_subject.append(fourth_df_1['First_Subject'][row])
        second_subject.append(fourth_df_1['Second_Subject'][row])
        third_subject.append(fourth_df_1['Third_Subject'][row])
        fourth_subject.append(fourth_df_1['Fourth_Subject'][row])
        fifth_subject.append(li.find('a').get('href').split('/')[4])
        url.append(f"https://www.barnesandnoble.com{li.find('a').get('href')}")
      print(f"*{row}: {fourth_df_1['First_Subject'][row]}, {fourth_df_1['Second_Subject'][row]}, {fourth_df_1['Third_Subject'][row]}, {fourth_df_1['Fourth_Subject'][row]}")

    break

In [ ]:
fifth_df = pd.DataFrame({
    'First_Subject':first_subject,
    'Second_Subject':second_subject,
    'Third_Subject':third_subject,
    'Fourth_Subject':fourth_subject,
    'Fifth_subject':fifth_subject,
    'url':url
})

fifth_df = fifth_df.drop_duplicates(ignore_index = True)
fifth_df

In [ ]:
fifth_df.to_csv('fifth_df.csv', index = False)

In [3]:
first, second, third, fourth,fifth,sixth, url =[], [],[],[],[],[],[]

In [ ]:
for row in fifth_df.index:
  if fifth_df['Fifth_subject'][row] == '_':
    first.append(fifth_df['First_Subject'][row])
    second.append(fifth_df['Second_Subject'][row])
    third.append(fifth_df['Third_Subject'][row])
    fourth.append(fifth_df['Fourth_Subject'][row])
    fifth.append(fifth_df['Fifth_subject'][row])
    sixth.append('_')
    url.append(fifth_df['url'][row])
    print(row)
    continue

  soup = crawler(fifth_df['url'][row])
  sidebar_sections = soup.find_all('div',class_='sidebar__section refinements')

  if sidebar_sections == []:
    first.append(fifth_df['First_Subject'][row])
    second.append(fifth_df['Second_Subject'][row])
    third.append(fifth_df['Third_Subject'][row])
    fourth.append(fifth_df['Fourth_Subject'][row])
    fifth.append(fifth_df['Fifth_subject'][row])
    sixth.append('_')
    url.append(fifth_df['url'][row])
    print(row)
    continue

  if sidebar_sections[0].find_all('li',class_='bg-gray-02 pt-xs pb-xs')!=[]:
      continue

  if str(sidebar_sections).find('Subjects')==-1:
    first.append(fifth_df['First_Subject'][row])
    second.append(fifth_df['Second_Subject'][row])
    third.append(fifth_df['Third_Subject'][row])
    fourth.append(fifth_df['Fourth_Subject'][row])
    fifth.append(fifth_df['Fifth_subject'][row])
    sixth.append('_')
    url.append(fifth_df['url'][row])
    print(row)

  for sidebar_section in sidebar_sections:
    if sidebar_section.find('span', class_='augHeader').text == "\n Subjects\n\n":
      for li in sidebar_section.find_all('li'):
        first.append(fifth_df['First_Subject'][row])
        second.append(fifth_df['Second_Subject'][row])
        third.append(fifth_df['Third_Subject'][row])
        fourth.append(fifth_df['Fourth_Subject'][row])
        fifth.append(fifth_df['Fifth_subject'][row])
        sixth.append(li.find('a').get('href').split('/')[4])
        url.append(f"https://www.barnesandnoble.com{li.find('a').get('href')}")
      print(f"*{row}: {fifth_df['First_Subject'][row]}, {fifth_df['Second_Subject'][row]}, {fifth_df['Third_Subject'][row]}, {fifth_df['Fourth_Subject'][row]}, {fifth_df['Fifth_subject'][row]}")

    break

In [ ]:
sixth_df = pd.DataFrame({
    'First_Subject':first,
    'Second_Subject':second,
    'Third_Subject':third,
    'Fourth_Subject':fourth,
    'Fifth_Subject':fifth,
    'Sixth_Subject':sixth,
    'url':url
})

sixth_df = sixth_df.drop_duplicates(ignore_index = True)
sixth_df

In [7]:
sixth_df.to_csv('sixth_df.csv',index=False)